In [84]:
import os
import json
import pandas as pd
import logging
import argparse

In [162]:
def data2df(data):
    from qiime2 import Metadata
    return data.view(Metadata).to_dataframe()

In [109]:
outdirs = ['/mnt/d/Yangk/work/git/amplicon_qiime2/test_out/SRR18505774/',
           '/mnt/d/Yangk/work/git/amplicon_qiime2/test_out/SRR18505775/',
           '/mnt/d/Yangk/work/git/amplicon_qiime2/test_out/SRR18505770/',
           '/mnt/d/Yangk/work/git/amplicon_qiime2/test_out/SRR18505774_silva/'
          ]

names = [i.split('/')[-2]  for i in outdirs]

jsons = [os.path.join(i, 'Amplicon_fq2asv.json') for i in outdirs]


In [131]:
def get_qza_list(json_list, qza_name='asv_rep_seqs_qza'):
    from qiime2 import Artifact
    qza_list = []
    for j in jsons:
        with open(j,'rt') as h:
            fq2asv = json.load(h)
            qza_list.append(Artifact.load(fq2asv[qza_name]))
    return qza_list

def merge_qza(qza_list, outfile, para='seq'):
    from qiime2.plugins.feature_table.methods import merge
    from qiime2.plugins.feature_table.methods import merge_seqs
    from qiime2.plugins.feature_table.methods import merge_taxa
    from qiime2 import Metadata
    
    merge_para = None
    if para == 'tab':
        merge_para = merge
        merged = merge_para(qza_list)
        merged.merged_table.save(outfile)
        return merged.merged_table.view(Metadata).to_dataframe().T
    elif para == 'seq':
        merge_para = merge_seqs
    elif para == 'taxa':
        merge_para = merge_taxa
    else:
        raise(f'{para} not exists in [tab, seq, taxa]')
    if merge_para:
        merged = merge_para(qza_list)
        merged.merged_data.save(outfile)
        return merged.merged_data.view(Metadata).to_dataframe()

In [136]:
def generate_seq_tree(rep_seq_qza):
    import qiime2.plugins.phylogeny.actions as phylogeny_actions

    action_results = phylogeny_actions.align_to_tree_mafft_fasttree(
        sequences=rep_seq_qza,
    )
    aligned_rep_seqs = action_results.alignment
    masked_aligned_rep_seqs = action_results.masked_alignment
    unrooted_tree = action_results.tree
    rooted_tree = action_results.rooted_tree
    return unrooted_tree, rooted_tree

rep_seqs = Artifact.load('test_out/merged_seq.qza')
unrooted_tree, rooted_tree = generate_seq_tree(rep_seqs)


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2-archive-oo4gdvnp/4d233265-149c-4ecd-bc85-e4d3bebb8b30/data/dna-sequences.fasta



inputfile = orig
61 x 250 - 73 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 61 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
   50 / 61
done.

Progressive alignment 1/2... 
STEP    33 / 60 (thread    0) f
Reallocating..done. *alloclen = 1501
STEP    60 / 60 (thread    0) f
done.

Making a distance matrix from msa.. 
    0 / 61 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
   50 / 61
done.

Progressive alignment 2/2... 
STEP    38 / 60 (thread    0) f
Reallocating..done. *alloclen = 1501
STEP    60 / 60 (thread    0) f
done.

disttbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --aut

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-idtve5vx/e4c1db4e-54a7-4910-bf9c-bf9ae5717ac7/data/aligned-dna-sequences.fasta



FastTree Version 2.1.10 Double precision (No SSE3)
Alignment: /tmp/qiime2-archive-idtve5vx/e4c1db4e-54a7-4910-bf9c-bf9ae5717ac7/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
Initial topology in 0.01 seconds
Refining topology: 24 rounds ME-NNIs, 2 rounds ME-SPRs, 12 rounds ML-NNIs
Total branch-length 4.732 after 0.05 sec
ML-NNI round 1: LogLk = -4945.246 NNIs 14 max delta 7.18 Time 0.08
Switched to using 20 rate categories (CAT approximation)
Rate categories were divided by 0.869 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -4365.246 NNIs 5 max delta 0.95 Time 0.10
ML-NNI round 3: LogLk = -4364.570 NNIs 2 max delta 0.00 Ti

In [156]:
def get_diversity(sample_meta, table, tree):
    import qiime2.plugins.diversity.actions as diversity_actions
    action_results = diversity_actions.core_metrics_phylogenetic(
        phylogeny=rooted_tree,
        table=table,
        sampling_depth=10000,
        metadata=sample_meta,
    )
    return action_results

table = Artifact.load('test_out/merged_table.qza')
action_results = get_diversity(sample_meta, table, rooted_tree)


/home/yangk/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [214]:
alpha_list = [action_results.faith_pd_vector, action_results.observed_features_vector, 
              action_results.shannon_vector, action_results.evenness_vector]
df_alpha_vector = pd.concat([i.view(pd.Series) for i in alpha_list], axis=1)

In [221]:
action_results.jaccard_distance_matrix.export_data('test_out/jaccard_distance_matrix/')

In [159]:
action_results.rarefied_table.view(Metadata).to_dataframe().sum(axis=1)

id
SRR18505774          10000.0
SRR18505775          10000.0
SRR18505770          10000.0
SRR18505774_silva    10000.0
dtype: float64

In [163]:
data2df(action_results.faith_pd_vector)

,faith_pd
Sample ID,
SRR18505774,4.711365
SRR18505775,4.280283
SRR18505770,3.872025
SRR18505774_silva,4.960432


In [205]:
action_results.faith_pd_vector.view(pd.Series)

SRR18505774          4.711365
SRR18505775          4.280283
SRR18505770          3.872025
SRR18505774_silva    4.960432
Name: faith_pd, dtype: float64

In [204]:
action_results.unweighted_unifrac_pcoa_results.view()

Exception: No transformation from <class 'q2_types.ordination._format.OrdinationDirectoryFormat'> to <class 'pandas.core.frame.DataFrame'>

In [211]:
action_results.unweighted_unifrac_pcoa_results.view(pd.DataFrame)

Exception: No transformation from <class 'q2_types.ordination._format.OrdinationDirectoryFormat'> to <class 'pandas.core.frame.DataFrame'>

In [192]:
action_results.unweighted_unifrac_pcoa_results.export_data('test_out/unweighted_unifrac_pcoa_results')

In [206]:
action_results.jaccard_pcoa_results.export_data('test_out/jaccard_pcoa_results')

In [210]:
action_results.jaccard_emperor.export_data('test_out/jaccard_emperor/')

In [173]:
unweighted_unifrac_distance_matrix = action_results.unweighted_unifrac_distance_matrix

In [179]:
sample_meta.get_ids()

{'SRR18505770', 'SRR18505774', 'SRR18505774_silva', 'SRR18505775'}

In [183]:
df_meta = pd.DataFrame(names)
df_meta.columns=['sample-id']
df_meta['sample'] = df_meta['sample-id']

In [186]:
df_meta.to_csv('test_out/tmpmanifest', sep='\t', index=False)

In [187]:
sample_meta = Metadata.load('test_out/tmpmanifest')

In [120]:
seq_qza_list = get_qza_list(jsons, qza_name='asv_rep_seqs_qza')

outfile = 'test_out/merged_seq.qza'
merged_seq = merge_qza(seq_qza_list, outfile=outfile, para='seq')

In [119]:
seq_qza_list = get_qza_list(jsons, qza_name='asv_table_qza')

outfile = 'test_out/merged_tax.qza'
merged_tax = merge_qza(seq_qza_list, outfile=outfile, para='taxa')

In [127]:
# def get_qza_list(json_list, qza_name='asv_rep_seqs_qza'):
from qiime2 import Artifact
qza_list = []
for j in jsons:
    with open(j,'rt') as h:
        fq2asv = json.load(h)
        qza_list.append(Artifact.load(os.path.join(os.path.split(fq2asv['asv_rep_seqs_qza'])[0], 'asv_table.qza')))
# return qza_list
# seq_qza_list = get_qza_list(jsons, qza_name='asv_table_qza')

outfile = 'test_out/merged_table.qza'
merged_tab = merge_qza(qza_list, outfile=outfile, para='tab')

In [137]:
merged_tab.sum()

id
SRR18505774          34220.0
SRR18505775          56727.0
SRR18505770          13388.0
SRR18505774_silva    34220.0
dtype: float64